## Coletor da página de busca do Youtube
Precisamos de pegar a página que retorna os resultados da busca (entra com as keywords e faz a ordenação pelos mais recentes) e precisamos de pegar a página de cada um dos vídeos retornados a partir da busca

In [ ]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4 #beautiful soup

In [ ]:
queries = ['computer+organization+and+architecture', 'computer+organization', 'computer+architecture']
url = 'https://www.youtube.com/results?search_query={query}&sp=CAI%253D&p={page}' #outra opção: utilizar selenium

### Coleta dos dados

In [ ]:
for query in queries:
    for page in range(1,101):
        formatted_url = url.format(query=query, page=page)
        print(formatted_url)
        response = rq.get(formatted_url) #GET request
        
        with open('./dados_brutos/{}_{}.html'.format(query, page), 'w+') as output:
            output.write(response.text)
        time.sleep(2)

## Extrator de dados da página de busca do Youtube

### Processamento dos dados brutos

In [ ]:
for query in queries:
    for page in range(1,101):
        with open('./dados_brutos/{}_{}.html'.format(query, page), 'r+') as data:
            page_html = data.read()
            
            parsed = bs4.BeautifulSoup(page_html)
            
            #encontra todas as tags com o nome passado por parâmetro. Passamos 'a' pois são as tags link
            tags = parsed.findAll('a')
            
            for instance in tags:
                # aria-describedby: encontra inspecionando o elemento do link; pegamos ele pois sabemos que se trata de um vídeo
                if instance.has_attr('aria-describedby'):
                    link = instance['href']
                    title = instance['title']
                    with open('parsed_videos.json', 'a+') as output: # a+: adiciona linhas ao arquivo
                        video_info = {'link': link, 'title': title, 'query': query}
                        output.write('{}\n'.format(json.dumps(video_info)))           

### Verificação do resultado

In [ ]:
df = read_json('parsed_videos.json', lines=True)

In [ ]:
df.sort_values('title')